In [31]:
import pandas as pd
import numpy as np
import pickle

In [36]:
#loading up my data from deferred revenue
import_thing = pickle.load(open('../data/processed/initial_forecast.p', 'rb'))
df_fcst = import_thing['forecast']
df_billings = import_thing['billings']
df_billings['is_forecast']= 0
df_fcst['is_forecast']=1
df = pd.concat([df_billings, df_fcst],
            join='outer',
            ignore_index=True)
df = df.fillna(0)
df.sort_values(by=['curr', 'BU', 'period'], inplace=True)

# save the final dataframe
#pickle.dump( df, open( "data/processed/final_forecast.p", "wb" ) )

In [37]:
df.head(10)

,curr,BU,period,recognized_DC,recognized_US,service_DC,service_US,deferred_B_DC,deferred_B_US,deferred_1M_DC,...,deferred_6M_US,deferred_1Y_US,deferred_2Y_US,deferred_3Y_US,Period_Weeks,is_forecast,r_squared,intercept,coeff,X_length
0,ARS,Creative,2017-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
1,ARS,Creative,2017-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
2,ARS,Creative,2017-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
3,ARS,Creative,2017-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
4,ARS,Creative,2017-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
5,ARS,Creative,2017-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
6,ARS,Creative,2017-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
7,ARS,Creative,2017-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
8,ARS,Creative,2017-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0
9,ARS,Creative,2018-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0


In [33]:
input_dict = pickle.load(open('/data/processed/final_forecast.p', 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: '/data/processed/final_forecast.p'

In [8]:
input_dict.keys()


dict_keys(['model_dict', 'billings', 'ADBE_cal', 'bookings', 'FX_forwards', 'FX_rates', 'forecast'])

In [9]:
df = input_dict['billings']
dff= df[(df['curr']=='USD')&
       (df['BU']=='Creative')]

In [10]:
dff.head(10)

,curr,BU,period,recognized_DC,recognized_US,service_DC,service_US,deferred_B_DC,deferred_B_US,deferred_1M_DC,...,deferred_1Y_DC,deferred_2Y_DC,deferred_3Y_DC,deferred_1M_US,deferred_3M_US,deferred_6M_US,deferred_1Y_US,deferred_2Y_US,deferred_3Y_US,Period_Weeks
3650,USD,Creative,2015-01,14559163.16,14559163.16,10000.00,10000.00,1500.00,1500.00,3.530601e+07,...,3.025592e+07,113022.580000,0.0,3.530601e+07,541969.585000,61250.000000,2.994515e+07,113022.580000,0.0,4.0
3651,USD,Creative,2015-02,8046578.41,8046578.41,4650.00,4650.00,22500.00,22500.00,3.620015e+07,...,3.183496e+07,73599.907073,5400.0,3.620015e+07,357000.550000,176250.000000,3.115471e+07,73599.907073,5400.0,4.0
3652,USD,Creative,2015-03,39811644.45,39811644.45,413507.83,413507.83,28600.00,28600.00,4.936785e+07,...,6.893778e+07,194153.634545,80563.6,4.936785e+07,688713.532857,136875.196667,6.839394e+07,194153.634545,80563.6,5.0
3653,USD,Creative,2015-04,9526500.08,9526500.08,7193.13,7193.13,123000.00,123000.00,4.113099e+07,...,3.646002e+07,48730.570417,0.0,4.113099e+07,393329.300000,150000.000000,3.603506e+07,48730.570417,0.0,4.0
3654,USD,Creative,2015-05,6109074.10,6109074.10,320.16,320.16,3000.00,3000.00,4.072232e+07,...,4.144004e+07,64170.036154,332258.0,4.072232e+07,400516.400000,26812.500000,4.111867e+07,64170.036154,332258.0,4.0
3655,USD,Creative,2015-06,31105538.88,31105538.88,179021.88,179021.88,17700.16,17700.16,5.425408e+07,...,8.325723e+07,135264.688000,0.0,5.425408e+07,718412.815000,118519.570000,8.297888e+07,135264.688000,0.0,5.0
3656,USD,Creative,2015-07,9252809.45,9252809.45,-40033.75,-40033.75,0.00,0.00,4.468037e+07,...,5.182635e+07,117933.144400,0.0,4.468037e+07,566201.140000,181842.500000,5.164711e+07,117933.144400,0.0,4.0
3657,USD,Creative,2015-08,11088482.85,11088482.85,1803.07,1803.07,6000.00,6000.00,4.707799e+07,...,5.403825e+07,175487.244706,0.0,4.707799e+07,421472.260000,382726.422500,5.388063e+07,175487.244706,0.0,4.0
3658,USD,Creative,2015-09,20803993.44,20803993.44,371900.63,371900.63,15000.00,15000.00,6.008676e+07,...,1.210989e+08,239503.853125,0.0,6.008676e+07,885099.367500,99456.912500,1.149930e+08,239503.853125,0.0,5.0
3659,USD,Creative,2015-10,12608522.70,12608522.70,162575.00,162575.00,0.00,0.00,4.944253e+07,...,6.691861e+07,463741.807826,0.0,4.944253e+07,567832.367500,242346.450000,6.569829e+07,463741.807826,0.0,4.0


In [30]:
dff.columns


Index(['curr', 'BU', 'period', 'recognized_DC', 'recognized_US', 'service_DC',
       'service_US', 'deferred_B_DC', 'deferred_B_US', 'deferred_1M_DC',
       'deferred_3M_DC', 'deferred_6M_DC', 'deferred_1Y_DC', 'deferred_2Y_DC',
       'deferred_3Y_DC', 'deferred_1M_US', 'deferred_3M_US', 'deferred_6M_US',
       'deferred_1Y_US', 'deferred_2Y_US', 'deferred_3Y_US', 'Period_Weeks'],
      dtype='object')

In [21]:
colors = ['lightgrey']*len(dff)
print(colors)

['lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey']


In [22]:
colors[len(dff)-13]= 'peach'


In [23]:
print(colors)

['lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'peach', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey']


In [24]:
this_length = len(dff)
this_length

63

In [19]:
colors[this_length]

'lightgrey'

In [27]:
change_list = np.arange(this_length-24, this_length-12)
print(change_list)

[39 40 41 42 43 44 45 46 47 48 49 50]


In [28]:
for i in change_list:
    colors[i] = 'darkgrey'
    

In [29]:
colors

['lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'darkgrey',
 'darkgrey',
 'darkgrey',
 'darkgrey',
 'darkgrey',
 'darkgrey',
 'darkgrey',
 'darkgrey',
 'darkgrey',
 'darkgrey',
 'darkgrey',
 'darkgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey',
 'lightgrey']